# Modelling metabolism in SORGHUM

### Generating SORGHUM core model

In [11]:
import pandas as pd
from cobra import io
from cobra.core import Metabolite, Reaction
from cobra import flux_analysis
from Functions import *
from IPython import display
import logging
logging.basicConfig()
logger = logging.getLogger('logger')

#import sbml file
fname = "C:\\Users\\sanus\\Scripts\\git\\plantcoremetabolism-model\\PlantCoreMetabolism_v1_3_2.xml"
model = io.read_sbml_model(fname)
model.solver='glpk'
#display.clear_output()

from sweetlovegroup import transform
model = transform.fixModelCompatibilityIssueCobra015(model,fname)


#change Biomass_tx to Protein_biomass
rxn = model.reactions.get_by_id("Biomass_tx")
rxn.id = "Protein_biomass_tx"
for met in ["K_b","Ca_b","Mg_b"]:
    met  = model.metabolites.get_by_id(met)
    coeff = rxn.metabolites.get(met)
    rxn.add_metabolites({met:-1*coeff})
met = Metabolite("Protein_b",name="Protein_biomass")
formula_dict = rxn.check_mass_balance()
met.formula = "".join([atom+str(formula_dict[atom]*-1) for atom in formula_dict.keys() if atom != "charge"])
met.charge = formula_dict["charge"]*-1
met.compartment = "b"
rxn.add_metabolites({met:1})

#change Biomass_tx to Protein_biomass
rxn = model.reactions.get_by_id("AraCore_Biomass_tx")
rxn.remove_from_model()


for rxn in model.reactions:
    if rxn.lower_bound == -1000:
        rxn.lower_bound = -3000
    if rxn.upper_bound == 1000:
        rxn.upper_bound = 3000


model.metabolites.get_by_id("aL_1_PHOSPHATIDYL_GLYCEROL_P_m").remove_from_model()
model.metabolites.get_by_id("aL_1_PHOSPHATIDYL_GLYCEROL_P_p").remove_from_model()

model.reactions.get_by_id("PGPPHOSPHA_RXN_m").add_metabolites({model.metabolites.get_by_id("L_1_PHOSPHATIDYL_GLYCEROL_P_m"):-0.03,
                                                               model.metabolites.get_by_id("PROTON_m"):-0.03})
model.reactions.get_by_id("PHOSPHAGLYPSYN_RXN_m").add_metabolites({model.metabolites.get_by_id("L_1_PHOSPHATIDYL_GLYCEROL_P_m"):0.03,
                                                                   model.metabolites.get_by_id("PROTON_m"):0.03})
model.reactions.get_by_id("PGPPHOSPHA_RXN_p").add_metabolites({model.metabolites.get_by_id("L_1_PHOSPHATIDYL_GLYCEROL_P_p"):-0.03,
                                                               model.metabolites.get_by_id("PROTON_p"):-0.03})
model.reactions.get_by_id("PHOSPHAGLYPSYN_RXN_p").add_metabolites({model.metabolites.get_by_id("L_1_PHOSPHATIDYL_GLYCEROL_P_p"):0.03,
                                                                   model.metabolites.get_by_id("PROTON_p"):0.03})
model.reactions.get_by_id("LPG_biosynthesis_c").add_metabolites({model.metabolites.get_by_id("L_1_PHOSPHATIDYL_GLYCEROL_P_p"):-0.03})

ERROR:cobra.io.sbml:No objective coefficients in model. Unclear what should be optimized


In [12]:
model.metabolites.get_by_id("L_1_PHOSPHATIDYL_GLYCEROL_P_m").charge

-3.0

In [13]:
import pandas as pd

df = pd.read_csv("Data/biomass_sorghum.csv")

FA=["PALMITATE_p","CPD_9245_p","CPD_17412_p","CPD_17291_p","STEARIC_ACID_p","OLEATE_CPD_p",
    "Octadecadienoate_p","LINOLENIC_ACID_p","ARACHIDIC_ACID_p","CPD_16709_p","DOCOSANOATE_p"]
FACP = {"PALMITATE_p":"Palmitoyl_ACPs_p",
        "CPD_9245_p":"Palmitoleoyl_ACP_p",
        "CPD_17412_p":"hexadecadienoate_ACP_p",
        "CPD_17291_p":"hexadecatrienoate_ACP_p",
        "STEARIC_ACID_p":"Stearoyl_ACPs_p",
        "OLEATE_CPD_p":"Oleoyl_ACPs_p",
        "Octadecadienoate_p":"Octadecadienoyl_ACP_p",
        "LINOLENIC_ACID_p":"Octadecatrienoyl_ACP_p",
        "ARACHIDIC_ACID_p":"Arachidoyl_ACPs_p",
        "CPD_16709_p":"Eicosenoyl_ACP_p",
        "DOCOSANOATE_p":"Behenoyl_ACPs_p"}


PLs = ["ACYL_SN_GLYCEROL_3P_p",
       "L_PHOSPHATIDATE_p","L_PHOSPHATIDATE_m","DIACYLGLYCEROL_p",
       "DIACYLGLYCEROL_r","Triacylglycerols_p","PHOSPHATIDYL_CHOLINE_r",
       "L_1_PHOSPHATIDYL_ETHANOLAMINE_r","L_1_PHOSPHATIDYL_GLYCEROL_p",
       "L_1_PHOSPHATIDYL_GLYCEROL_P_p","L_1_PHOSPHATIDYL_GLYCEROL_P_m",
       "L_1_PHOSPHATIDYL_GLYCEROL_m","2_Lysophosphatidylcholines_r",
       "Lysophosphatidylglycerols_r",]


for met in PLs:
    met=model.metabolites.get_by_id(met)
    met.formula=""

### Use biomass composition to generate biomass equations

In [14]:
def generateMissingFormula(model,debug=False):
    loop_counter = 0
    former = 0
    for met in model.metabolites:
        if met.formula == "" or met.formula == "NA":
            former = former +1
    latter = 1
    while True:
        loop_counter = loop_counter+1
        if debug:
            print("Loop = "+str(loop_counter))
        former = latter
        for rxn in model.reactions:
            count = 0
            for met in rxn.metabolites:
                if met.formula=="" or met.formula=="NA" or met.formula == None:
                    if met.formula == "NA" or met.formula == None:
                        met.formula = ""
                    count = count + 1
            if count == 1:
                unb = rxn.check_mass_balance()
                eqn = rxn.reaction
                eqn = " "+eqn+" "
                for met in rxn.metabolites.keys():
                    formula = met.formula
                    if formula == None:
                        formula = "0"
                        NF_list.add(rxn.id)
                    eqn=eqn.replace(" "+met.id+" ","("+formula+")")
                if debug:
                    print(eqn)
                    print(unb)
                for met in rxn.metabolites:
                    if met.formula == "":
                        tempForm = ""
                        for a in sorted(unb.keys()):
                            if a=="charge" or round(unb[a],2)==0:
                                continue
                            num = float(abs(unb[a]))
                            if str(round(num))==str(num):
                                tempForm = tempForm+a+str(int(round(num)))
                            else:
                                tempForm = tempForm+a+str(num)
                                #print(a)
                                #print(round(num)==num)
                                #print(round(num))
                                #print(num)
                                #print(type(round(num)))
                                #print(type(num))
                        met.formula = tempForm
                        if debug:
                            print(met.id)
                            print(tempForm)
        latter = 0
        for met in model.metabolites:
            if met.formula == "" or met.formula == "NA":
                latter = latter +1
        if former == latter:
            break
        
        

### Run stem

In [15]:
from cobra import flux_analysis
stem_model = model.copy()

k = "stem"
RXN1 = Reaction("Fatty_acid_mix_"+k)
RXN2 = Reaction("Fatty_acid_ACP_"+k)
tot = 0
for met in df["Unnamed: 0"]:
    #print met
    if met in FA:
        RXN1.add_metabolites({stem_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])})
        RXN2.add_metabolites({stem_model.metabolites.get_by_id(FACP[met]):-1*float(df[df["Unnamed: 0"]==met][k])})
        tot = tot+(float(df[df["Unnamed: 0"]==met][k]))
if tot==0:
    RXN1.add_metabolites({stem_model.metabolites.PALMITATE_p:-1})
    RXN2.add_metabolites({stem_model.metabolites.Palmitoyl_ACPs_p:-1})
    tot = 1
RXN1.add_metabolites({stem_model.metabolites.Fatty_Acids_p:tot})
RXN1.lower_bound = 1000
RXN1.upper_bound = 0
stem_model.add_reaction(RXN1)

RXN2.add_metabolites({stem_model.metabolites.Fatty_acyl_ACP_p:tot})
RXN2.lower_bound = 1000
RXN2.upper_bound = 0
stem_model.add_reaction(RXN2)

generateMissingFormula(stem_model)

rxn = Reaction("Biomass_stem_tx")
for met in df["Unnamed: 0"]:
    if met in FA:
        continue
    rxn.add_metabolites({stem_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])})
rxn.lower_bound = 0
rxn.upper_bound = 1000
stem_model.add_reaction(rxn)


stem_model.reactions.GLC_tx.upper_bound = 0
stem_model.reactions.GLC_tx.lower_bound = 0
stem_model.reactions.Sucrose_tx.upper_bound = 1
stem_model.reactions.Sucrose_tx.lower_bound = 1
stem_model.reactions.Photon_tx.upper_bound = 0
stem_model.reactions.Photon_tx.lower_bound = 0
stem_model.reactions.Biomass_stem_tx.objective_coefficient=1

flux_analysis.parsimonious.pfba(stem_model)
print("Biomass flux ="+str(stem_model.reactions.Biomass_stem_tx.flux))

Biomass flux =0.3965874837727823


In [16]:
stem_model.reactions.query("Biomass_tx")

[]

In [17]:
stem_model.reactions.Biomass_stem_tx.reaction

'0.728315974933178 COUMARATE_c + 1.7764240905725708 Cellulose_b + 0.38776720411991217 FRU_c + 0.4602470553572778 GLC_c + 0.8994069662814456 Xylan_b + 0.1068123162394869 sSUCROSE_b --> '

### Run root

In [18]:
from cobra import flux_analysis
root_model = model.copy()

k = "root"
RXN1 = Reaction("Fatty_acid_mix_"+k)
RXN2 = Reaction("Fatty_acid_ACP_"+k)
tot = 0
for met in df["Unnamed: 0"]:
    #print met
    if met in FA:
        RXN1.add_metabolites({root_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])})
        RXN2.add_metabolites({root_model.metabolites.get_by_id(FACP[met]):-1*float(df[df["Unnamed: 0"]==met][k])})
        tot = tot+(float(df[df["Unnamed: 0"]==met][k]))
if tot==0:
    RXN1.add_metabolites({root_model.metabolites.PALMITATE_p:-1})
    RXN2.add_metabolites({root_model.metabolites.Palmitoyl_ACPs_p:-1})
    tot = 1
RXN1.add_metabolites({root_model.metabolites.Fatty_Acids_p:tot})
RXN1.lower_bound = 0
RXN1.upper_bound = 3000
root_model.add_reaction(RXN1)
    
RXN2.add_metabolites({root_model.metabolites.Fatty_acyl_ACP_p:tot})
RXN2.lower_bound = 0
RXN2.upper_bound = 3000
root_model.add_reaction(RXN2)


generateMissingFormula(root_model)

rxn = Reaction("Biomass_root_tx")
for met in df["Unnamed: 0"]:
    if met in FA:
        continue
    rxn.add_metabolites({root_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])})
rxn.lower_bound = 0
rxn.upper_bound = 1000
root_model.add_reaction(rxn)

root_model.reactions.GLC_tx.upper_bound = 0
root_model.reactions.GLC_tx.lower_bound = 0
root_model.reactions.Sucrose_tx.upper_bound = 1
root_model.reactions.Sucrose_tx.lower_bound = 1
root_model.reactions.Photon_tx.upper_bound = 0
root_model.reactions.Photon_tx.lower_bound = 0
root_model.reactions.Biomass_root_tx.objective_coefficient=1
flux_analysis.parsimonious.pfba(root_model)
print("Biomass flux ="+str(root_model.reactions.Biomass_root_tx.flux))

Biomass flux =0.10047059495464955


In [19]:
stem_model.reactions.Biomass_stem_tx.reaction

'0.728315974933178 COUMARATE_c + 1.7764240905725708 Cellulose_b + 0.38776720411991217 FRU_c + 0.4602470553572778 GLC_c + 0.8994069662814456 Xylan_b + 0.1068123162394869 sSUCROSE_b --> '

### Run seed

In [20]:
from cobra import flux_analysis
seed_model = model.copy()

k = "seed"
RXN1 = Reaction("Fatty_acid_mix_"+k)
RXN2 = Reaction("Fatty_acid_ACP_"+k)
tot = 0
for met in df["Unnamed: 0"]:
    #print met
    if met in FA:
        RXN1.add_metabolites({seed_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])})
        RXN2.add_metabolites({seed_model.metabolites.get_by_id(FACP[met]):-1*float(df[df["Unnamed: 0"]==met][k])})
        tot = tot+(float(df[df["Unnamed: 0"]==met][k]))
if tot==0:
    RXN1.add_metabolites({seed_model.metabolites.PALMITATE_p:-1})
    RXN2.add_metabolites({seed_model.metabolites.Palmitoyl_ACPs_p:-1})
    tot = 1
RXN1.add_metabolites({seed_model.metabolites.Fatty_Acids_p:tot})
RXN1.lower_bound = 0
RXN1.upper_bound = 0
seed_model.add_reaction(RXN1)
    
RXN2.add_metabolites({seed_model.metabolites.Fatty_acyl_ACP_p:tot})
RXN2.lower_bound = 0
RXN2.upper_bound = 0
seed_model.add_reaction(RXN2)


generateMissingFormula(seed_model)

rxn = Reaction("Biomass_seed_tx")
for met in df["Unnamed: 0"]:
    if met in FA:
        continue
    rxn.add_metabolites({seed_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])})
rxn.lower_bound = 0
rxn.upper_bound = 1000
seed_model.add_reaction(rxn)

seed_model.reactions.GLC_tx.upper_bound = 0
seed_model.reactions.GLC_tx.lower_bound = 0
seed_model.reactions.Sucrose_tx.upper_bound = 1
seed_model.reactions.Sucrose_tx.lower_bound = 1
seed_model.reactions.Photon_tx.upper_bound = 0
seed_model.reactions.Photon_tx.lower_bound = 0
seed_model.reactions.Biomass_seed_tx.objective_coefficient=1
sol=flux_analysis.parsimonious.pfba(seed_model)
print("Biomass flux ="+str(seed_model.reactions.Biomass_seed_tx.flux))

    
    

Biomass flux =0.7498501513695357


In [21]:
seed_model.reactions.Biomass_seed_tx.reaction

'0.0290287408345124 ARG_c + 0.04964753924729917 COUMARATE_c + 0.009526544142541593 CYS_c + 0.08847893279975096 FRU_c + 0.09180938196410796 GLC_c + 0.05121597854438592 GLY_c + 0.0185023834727556 HIS_c + 0.043587502664421914 ILE_c + 0.14186704529664612 LEU_c + 0.019181547669595882 LYS_c + 0.013842921277520416 MET_c + 0.04418703765877103 PHE_c + 0.0980210478906567 PRO_c + 0.03573277660229751 THR_c + 0.0066399337567809545 TRP_c + 0.03001851230031804 TYR_c + 0.060842258637074514 VAL_c + 0.14252283395878335 sALA_b + 0.5513719420874239 sASP_b + 1.4793643163548391 sGLU_b + 0.05833794751890824 sSER_b + 0.012094778187610927 sSUCROSE_b --> '

### Generate diel leaf model



In [22]:

def generateMultiphaseModel(model,tags):
    models=dict()
    for i in range(0,len(tags)):
        temp=model.copy()
        for met in temp.metabolites:
            met.id = met.id+str(i+1)
            met.compartment = met.compartment+str(i+1)
            met.name = met.name+" "+tags[i]
        for rxn in temp.reactions:
            rxn.id = rxn.id+str(i+1)
            rxn.name = rxn.name+" "+tags[i]
        if i==0:
            modelOut = temp.copy()
        else:
            modelOut = modelOut+temp
            for met in temp.metabolites:
                if met.id not in modelOut.metabolites:
                    met2 = met.copy()
                    #print(met2.id)
                    modelOut.add_metabolites(met2)
        for comp in temp.compartments:
            modelOut.compartments={str(comp):model.compartments[comp[0:len(comp)-1]]+" "+tags[i]}
    return modelOut


def addLinkers(model,baseMets,from_phase,to_phase,reversible=True):
    from cobra.core import Reaction
    for met in baseMets.keys():
        baseMet = met
    rxn = Reaction(baseMet+"_linker"+from_phase+to_phase)
    rxn.name = rxn.id
    baseMets = baseMets[baseMet]
    for baseMetID in baseMets.keys():
        met1 = model.metabolites.get_by_id(baseMetID+from_phase)
        met2 = model.metabolites.get_by_id(baseMetID+to_phase)
        rxn.add_metabolites({met1:-1*baseMets[baseMetID],met2:1*baseMets[baseMetID]})
    rxn.upper_bound = 1000
    if reversible:
        rxn.lower_bound = -1000
    else:
        rxn.lower_bound = 0
    model.add_reaction(rxn)
    return model
    

def identifyFractionalMets(model):
    import re
    fractionMets=dict()
    for rxn in model.reactions:
        for met in rxn.metabolites.keys():
            prefix=""
            a=re.search("^a{1,3}",met.id)
            anion=""
            if a:
                anion=a.group(0)
                prefix=anion
            b=re.search("^b{1,3}",met.id)
            basic=""
            if b:
                basic=b.group(0)
                prefix=basic
            if (not prefix == ""):
                mainMet=model.metabolites.get_by_id(met.id[len(prefix):])
                fractionMets[met.id]=mainMet.id
                #fractionMets[met.id]=prefix
    return fractionMets


# def mergeCompartments(model,comp1,comp2):
#     temp = model.copy()
#     for met in model.metabolites:
#         if met.compartment==comp2:
#             M_orig = met.id.replace(comp2,comp1)
#             if M_orig in model.metabolites:
#                 M_orig = model.metabolites.get_by_id(M_orig)
#                 for rxn in M_orig.reactions:
#                     M2 = model.metabolites.get_by_id(M_orig.id)
#                     coeff = rxn.metabolites[M_orig]
#                     rxn.add_metabolites({M_orig:-1*coeff,M2:coeff})
                    
def addUpperAndLowerBoundsFromFile(model,filename):
    import pandas as pd
    df = pd.read_csv(filename,sep="\t")
    
    for rxn in df["reaction"]:
        rxn = model.reactions.get_by_id(str(rxn))
        rxn.lower_bound = float(df[df["reaction"]==rxn.id]["lower-bound"])
        rxn.upper_bound = float(df[df["reaction"]==rxn.id]["upper-bound"])
    
    return model


def generateC4leafModel(model,constraintsFile):
    tags = ["mesophyll day","mesophyll night","bundlesheath day","bundlesheath night"]
    C4_model = generateMultiphaseModel(model,tags)

    mets1 = ["STARCH_p","MAL_v","FUM_v","CIT_v","NITRATE_v","SUCROSE_v","GLC_v","FRU_v"]
    for met in mets1:
        if met == "MAL_v":
            mets = {met:{"MAL_v":0.7,"aMAL_v":0.3}}
        elif met == "FUM_v":
            mets = {met:{"FUM_v":0.92,"aFUM_v":0.08}}
        elif met == "CIT_v":
            mets = {met:{"CIT_v":0.5,"aCIT_v":0.5}}
        else:
            mets = {met:{met:1}}
        C4_model = addLinkers(C4_model,mets,"1","2")
        C4_model = addLinkers(C4_model,mets,"3","4")

    mets2 = ["ASN_v","GLT_v","L_ALPHA_ALANINE_v","LEU_v","bHIS_v","MET_v","LYS_v","THR_v","VAL_v","PHE_v",
            "4_AMINO_BUTYRATE_v","SER_v","ARG_v","GLN_v","GLY_v","L_ASPARTATE_v","ILE_v","PRO_v","CYS_v",
            "TRP_v","TYR_v"]
    for met in mets2:
        mets = {met:{met:1}}
        C4_model = addLinkers(C4_model,mets,"1","2",reversible=False)
        C4_model = addLinkers(C4_model,mets,"3","4",reversible=False)


    mets3 = ["MAL_c","PYRUVATE_c","ASN_c","GLT_c","L_ALPHA_ALANINE_c","LEU_c",
            "MET_c","LYS_c","HIS_c","THR_c","VAL_c","PHE_c","4_AMINO_BUTYRATE_c","SER_c","ARG_c","GLN_c",
            "GLY_c","L_ASPARTATE_c","ILE_c","PRO_c","CYS_c","TRP_c","TYR_c","G3P_c","GAP_c","Pi_c",
            "PHOSPHO_ENOL_PYRUVATE_c"] #"SUCROSE_c","MALTOSE_p",
    for met in mets3:
        if met=="Pi_c":
            mets = {met:{"Pi_c":0.7,"aPi_c":0.3}}
        else:
            mets = {met:{met:1}}
        C4_model = addLinkers(C4_model,mets,"1","3")
        C4_model = addLinkers(C4_model,mets,"2","4")


    C4_model = addUpperAndLowerBoundsFromFile(C4_model,constraintsFile)   

    #Nitrate ratio day:night
    met = Metabolite("Nitrate_pseudo")
    met.name = "Nitrate_pseudo"
    met.compartment = "f"
    met.charge = 0
    for i in (1,3):
        C4_model.reactions.get_by_id(str("Nitrate_tx" + str(i))).add_metabolites({met : 2})
    for i in (2,4):
        C4_model.reactions.get_by_id(str("Nitrate_tx" + str(i))).add_metabolites({met : -3})

    #Rubisco oxygenase
    for i in range(1,5):
        met = Metabolite("Rubisco_pseudo" + str(i))
        met.name = "Rubisco_pseudo" + str(i)
        met.compartment = "f"
        met.charge = 0
        C4_model.reactions.get_by_id(str("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p" + str(i))).add_metabolites({met : 1})
        C4_model.reactions.get_by_id(str("RXN_961_p" + str(i))).add_metabolites({met : -20})


    #Maintainence costs
    for i in range(1,5):
        C4_model.reactions.get_by_id(str("ATPase_tx" + str(i))).upper_bound = C4_model.reactions.get_by_id(str("ATPase_tx" + str(i))).lower_bound = 19.5
        met = Metabolite(str("Maint_pseudo" + str(i)))
        met.name = str("Maint_pseudo" + str(i))
        met.compartment = "f"
        met.charge = 0
        C4_model.reactions.get_by_id(str("ATPase_tx" + str(i))).add_metabolites({met : 1})
        C4_model.reactions.get_by_id(str("NADPHoxc_tx" + str(i))).add_metabolites({met : -3})
        C4_model.reactions.get_by_id(str("NADPHoxm_tx" + str(i))).add_metabolites({met : -3})
        C4_model.reactions.get_by_id(str("NADPHoxp_tx" + str(i))).add_metabolites({met : -3})



    for i in range(3,5):
        for met in model.metabolites:
            if "CARBON_DIOXIDE" in met.id or "OXYGEN_MOLECULE" in met.id or "Photon" in met.id:
                continue
            if met.compartment == "e"+str(i):
                rxn = Reaction(met.id+"_e"+str(i-2)+str(i))
                rxn.add_metabolites({met:1,C4_model.metabolites.get_by_id(met.id.replace(met.compartment,"e"+str(i-2))):-1})




    met1 = Metabolite("X_Phloem_contribution_day",name="Phloem output during the day",compartment="b1")
    C4_model.reactions.get_by_id("Phloem_output_tx1").remove_from_model()
    C4_model.reactions.get_by_id("Phloem_output_tx3").add_metabolites({met1:1})
    met2 = Metabolite("X_Phloem_contribution_night",name="Phloem output during at night",compartment="b1")
    C4_model.reactions.get_by_id("Phloem_output_tx2").remove_from_model()
    C4_model.reactions.get_by_id("Phloem_output_tx4").add_metabolites({met2:1})

    rxn = Reaction("diel_biomass")
    rxn.add_metabolites({met1:-0.75,met2:-0.25})
    rxn.lower_bound = 0
    rxn.upper_bound = 1000
    C4_model.add_reaction(rxn)
    rxn.objective_coefficient=1
    return C4_model



In [23]:
leaf_model_source = generateC4leafModel(model,"Sorghum_Leaf_Constraints.txt")

leaf_model_source.reactions.Photon_tx1.upper_bound = 600
leaf_model_source.reactions.Photon_tx3.upper_bound = 400

sol = flux_analysis.parsimonious.pfba(leaf_model_source)
print("Phloem output = ")
print(leaf_model_source.reactions.diel_biomass.flux)

Phloem output = 
5.043363499140207


In [24]:
for i in range(1,5):
    for rxn in ["CO2_tx","O2_tx","PSII_RXN_p","RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p",
                "PEPCARBOXYKIN_RXN_c","1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m","MALIC_NADP_RXN_p",
                "MALIC_NADP_RXN_c"]:
        print(rxn+str(i)+" flux = "+str(sol.fluxes[rxn+str(i)]))
    print("------")

CO2_tx1 flux = 52.62912520685068
O2_tx1 flux = -71.74999999999991
PSII_RXN_p1 flux = 75.0
RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p1 flux = 0.0
PEPCARBOXYKIN_RXN_c1 flux = 0.0
1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m1 flux = 0.0
MALIC_NADP_RXN_p1 flux = 0.0
MALIC_NADP_RXN_c1 flux = 0.0
------
CO2_tx2 flux = -1.803662969025908
O2_tx2 flux = 4.552220817409059
PSII_RXN_p2 flux = 0.0
RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p2 flux = 0.0
PEPCARBOXYKIN_RXN_c2 flux = 0.0
1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m2 flux = 0.0
MALIC_NADP_RXN_p2 flux = 0.0
MALIC_NADP_RXN_c2 flux = 0.0
------
CO2_tx3 flux = 0.0
O2_tx3 flux = 8.059935646407002
PSII_RXN_p3 flux = 0.0
RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p3 flux = 64.13247528542666
PEPCARBOXYKIN_RXN_c3 flux = 0.0
1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m3 flux = 0.0
MALIC_NADP_RXN_p3 flux = 62.28896519259003
MALIC_NADP_RXN_c3 flux = 0.0
------
CO2_tx4 flux = 0.0
O2_tx4 flux = 7.252547355722501
PSII_RXN_p4 flux = 0.0
RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p4 flux = 0.0
PE

In [25]:
leaf_model = model.copy()


k = "leaf"
RXN1 = Reaction("Fatty_acid_mix_"+k)
RXN2 = Reaction("Fatty_acid_ACP_"+k)
tot = 0
for met in df["Unnamed: 0"]:
    #print met
    if met in FA:
        RXN1.add_metabolites({leaf_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])})
        RXN2.add_metabolites({leaf_model.metabolites.get_by_id(FACP[met]):-1*float(df[df["Unnamed: 0"]==met][k])})
        tot = tot+(float(df[df["Unnamed: 0"]==met][k]))
if tot==0:
    RXN1.add_metabolites({leaf_model.metabolites.PALMITATE_p:-1})
    RXN2.add_metabolites({leaf_model.metabolites.Palmitoyl_ACPs_p:-1})
    tot = 1
RXN1.add_metabolites({leaf_model.metabolites.Fatty_Acids_p:tot})
RXN1.lower_bound = 0
RXN1.upper_bound = 0
leaf_model.add_reaction(RXN1)
    
RXN2.add_metabolites({leaf_model.metabolites.Fatty_acyl_ACP_p:tot})
RXN2.lower_bound = 0
RXN2.upper_bound = 0
leaf_model.add_reaction(RXN2)


generateMissingFormula(leaf_model)

rxn = Reaction("Biomass_leaf_tx")
for met in df["Unnamed: 0"]:
    if met in FA:
        continue
    rxn.add_metabolites({leaf_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met]["stem"])})
rxn.lower_bound = 0
rxn.upper_bound = 1000
leaf_model.add_reaction(rxn)

leaf_model_sink = generateC4leafModel(leaf_model,"Sorghum_Leaf_Constraints.txt")


met = Metabolite("X_Biomass_contribution_day")
leaf_model_sink.reactions.Biomass_leaf_tx1.add_metabolites({met:1})
leaf_model_sink.reactions.Biomass_leaf_tx3.add_metabolites({met:1})
leaf_model_sink.metabolites.X_Phloem_contribution_day.remove_from_model()
leaf_model_sink.reactions.diel_biomass.add_metabolites({met:-0.75})

met = Metabolite("X_Biomass_contribution_night")
leaf_model_sink.reactions.Biomass_leaf_tx2.add_metabolites({met:1})
leaf_model_sink.reactions.Biomass_leaf_tx4.add_metabolites({met:1})
leaf_model_sink.metabolites.X_Phloem_contribution_night.remove_from_model()
leaf_model_sink.reactions.diel_biomass.add_metabolites({met:-0.25})

leaf_model_sink.reactions.diel_biomass.objective_coefficient=1
sol = flux_analysis.parsimonious.pfba(leaf_model_sink)
print("Biomass accumulation = ")
print(leaf_model_sink.reactions.diel_biomass.flux)

Biomass accumulation = 
11.495288565547526


In [26]:
leaf_model_sink.reactions.Biomass_leaf_tx4.reaction

'0.728315974933178 COUMARATE_c4 + 1.7764240905725708 Cellulose_b4 + 0.38776720411991217 FRU_c4 + 0.4602470553572778 GLC_c4 + 0.8994069662814456 Xylan_b4 + 0.1068123162394869 sSUCROSE_b4 --> X_Biomass_contribution_night'